[source](../../api/alibi_detect.cd.cvm_online.rst)

# Online Cramér-von Mises

## Overview

TODO - Describe.

## Usage

### Initialize


Arguments:

* `x_ref`: Data used as reference distribution.
* `ert`: The expected run-time (ERT) in the absence of drift.
* `window_size`: The sizes of the sliding test-windows used to compute the test-statistic. Smaller windows focus on responding quickly to severe drift, larger windows focus on ability to detect slight drift.


Keyword arguments:

TODO


Initialized drift detector example:


```python
from alibi_detect.cd import CVMDriftOnline

cd = CVMDriftOnline(x_ref, ert, window_size)
```

TODO - demonstration of use as supervised drift detector (on 1D loss stream), and for multivariate tabular data.

### Detect Drift

We detect data drift by sequentially calling `predict` on single instances `x_t` (no batch dimension) as they each arrive. We can return the test-statistic and the threshold by setting `return_test_stat` to *True*.

The prediction takes the form of a dictionary with `meta` and `data` keys. `meta` contains the detector's metadata while `data` is also a dictionary which contains the actual predictions stored in the following keys:

* `is_drift`: 1 if the test-window (of the most recent `window_size` observations) has drifted from the reference data and 0 otherwise.

* `time`: The number of observations that have been so far passed to the detector as test instances.

* `ert`: The expected run-time the detector was configured to run at in the absence of drift.

* `test_stat`: CVM test-statistic between the reference data and the test_window if `return_test_stat` equals *True*.

* `threshold`: The value the test-statsitic is required to exceed for drift to be detected if `return_test_stat` equals *True*.


```python
preds = cd.predict(x_t, return_test_stat=True)
```

Resetting the detector with the same reference data and thresholds but with a new and empty test-window is straight-forward:

```python
cd.reset()
```

## Examples

Coming soon!